<a href="https://colab.research.google.com/github/meizhong986/WhisperJAV/blob/main/notebook/WhisperJAV_colab_edition_expert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# WhisperJAV Colab Edition v1.7.4 (Expert)

| User Mode | What it does | Speed |
|------|--------------|-------|
| **Standard** | Processes your video once | Faster |
| **Two-Step** | Processes twice and combines for better accuracy | Slower |

| Option | What it controls |
|--------|------------------|
| **Scene Detection** | How to split audio into chunks (auditok, silero, semantic) |
| **Speech Segmenter** | How to detect speech in audio (silero, ten) |
| **Speech Enhancer** | Audio cleanup for noisy sources (ffmpeg-dsp, clearvoice, etc.) |
| **Model** | Which AI model to use (large-v2, large-v3, turbo, kotoba) |

---
<div style="font-size: 8px; line-height: 1.0;">
1. Upload your videos to <code>Google Drive/WhisperJAV/</code><br>
2. Run <b>Step 1: Expert Configuration</b><br>
3. Run <b>Step 2: Transcribe</b> and wait for completion<br>
4. Run <b>Step 3: AI Translation</b> (if selected)
</div>

<small>The notebook will automatically disconnect when finished to save your GPU credits.</small>

In [ ]:
#@title Step 1: Expert Configuration { display-mode: "form" }

#@markdown ## 📁 Files & Output
folder_name = "WhisperJAV" #@param {type:"string"}
subtitle_language = "Japanese" #@param ["Japanese", "English (auto-translate)", "English (AI translate)"]

#@markdown ---
#@markdown ## 1️⃣ Pass 1 Settings
pass1_pipeline = "balanced" #@param ["faster", "fast", "balanced", "fidelity", "transformers"]
pass1_sensitivity = "aggressive" #@param ["conservative", "balanced", "aggressive"]
pass1_model = "automatic" #@param ["automatic", "large-v2", "large-v3", "turbo", "kotoba-bilingual", "kotoba-v2.0", "kotoba-v2.1", "kotoba-v2.2"]

#@markdown **Expert Audio Setup**
pass1_scene_detector = "automatic" #@param ["automatic", "auditok", "silero", "semantic"]
pass1_speech_segmenter = "automatic" #@param ["automatic", "silero", "ten", "none"]
pass1_speech_enhancer = "none" #@param ["none", "ffmpeg-dsp", "clearvoice", "zipenhancer", "bs-roformer"]
#@markdown <font size="1">auditok=energy (fast), silero=VAD, semantic=texture (complex audio) | enhancer: ffmpeg-dsp(no GPU), clearvoice(48k), bs-roformer(vocal)</font>

#@markdown **FFmpeg Filters** *(only if enhancer is ffmpeg-dsp)*
pass1_ffmpeg_amplify = True #@param {type:"boolean"}
pass1_ffmpeg_loudnorm = False #@param {type:"boolean"}
pass1_ffmpeg_compress = False #@param {type:"boolean"}
pass1_ffmpeg_highpass = False #@param {type:"boolean"}

#@markdown ---
#@markdown ## 🔄 Two-Step Processing
use_two_step = False #@param {type:"boolean"}
merge_method = "prefer first step" #@param ["automatic", "keep all", "prefer first step", "prefer second step"]

#@markdown ---
#@markdown ## 2️⃣ Pass 2 Settings *(if enabled)*
pass2_pipeline = "transformers" #@param ["faster", "fast", "balanced", "fidelity", "transformers"]
pass2_sensitivity = "aggressive" #@param ["conservative", "balanced", "aggressive"]
pass2_model = "automatic" #@param ["automatic", "large-v2", "large-v3", "turbo", "kotoba-bilingual", "kotoba-v2.0", "kotoba-v2.1", "kotoba-v2.2"]

#@markdown **Expert Audio Setup**
pass2_scene_detector = "automatic" #@param ["automatic", "auditok", "silero", "semantic"]
pass2_speech_segmenter = "automatic" #@param ["automatic", "silero", "ten", "none"]
pass2_speech_enhancer = "none" #@param ["none", "ffmpeg-dsp", "clearvoice", "zipenhancer", "bs-roformer"]

#@markdown **FFmpeg Filters** *(only if enhancer is ffmpeg-dsp)*
pass2_ffmpeg_amplify = True #@param {type:"boolean"}
pass2_ffmpeg_loudnorm = False #@param {type:"boolean"}
pass2_ffmpeg_compress = False #@param {type:"boolean"}
pass2_ffmpeg_highpass = False #@param {type:"boolean"}

#@markdown ---
#@markdown ## 🤖 AI Translation *(if selected)*
translation_service = "local" #@param ["local", "deepseek", "openrouter", "gemini", "claude", "gpt"]
local_model = "gemma-9b" #@param ["gemma-9b", "llama-8b", "llama-3b", "auto"]
#@markdown <font size="1">local: Free, runs on GPU. gemma-9b (8GB+ VRAM), llama-8b (6GB+), llama-3b (3GB+). Cloud providers require API key.</font>
api_key = "" #@param {type:"string"}
translation_style = "standard" #@param ["standard", "explicit"]

#@markdown ---
#@markdown ## ⚙️ Session
opening_credit = "" #@param {type:"string"}
closing_credit = "Subs by WhisperJAV" #@param {type:"string"}
auto_disconnect = True #@param {type:"boolean"}

# ═══════════════════════════════════════════
# CONFIGURATION LOGIC
# ═══════════════════════════════════════════

# Venv path for WhisperJAV commands
VENV_PATH = "/content/whisperjav_env"
WHISPERJAV_CMD = f"{VENV_PATH}/bin/whisperjav"
WHISPERJAV_TRANSLATE_CMD = f"{VENV_PATH}/bin/whisperjav-translate"

# Mappings
combine_map = {"automatic": "smart_merge", "keep all": "full_merge",
               "prefer first step": "pass1_primary", "prefer second step": "pass2_primary"}
language_map = {"Japanese": "native", "English (auto-translate)": "direct-to-english",
                "English (AI translate)": "llm"}
tone_map = {"standard": "standard", "explicit": "pornify"}

model_map = {
    "automatic": None,
    "large-v2": "large-v2",
    "large-v3": "large-v3",
    "turbo": "large-v3-turbo",
    "kotoba-bilingual": "kotoba-tech/kotoba-whisper-bilingual-v1.0",
    "kotoba-v2.0": "kotoba-tech/kotoba-whisper-v2.0",
    "kotoba-v2.1": "kotoba-tech/kotoba-whisper-v2.1",
    "kotoba-v2.2": "kotoba-tech/kotoba-whisper-v2.2"
}

# Auto-correction
KOTOBA_MODELS = {"kotoba-bilingual", "kotoba-v2.0", "kotoba-v2.1", "kotoba-v2.2"}
LEGACY_PIPELINES = {"faster", "fast", "balanced", "fidelity"}
warnings_list = []

if pass1_model in KOTOBA_MODELS and pass1_pipeline in LEGACY_PIPELINES:
    warnings_list.append(f"Pass 1: {pass1_model} requires 'transformers' pipeline. Auto-correcting.")
    pass1_pipeline = "transformers"

if use_two_step and pass2_model in KOTOBA_MODELS and pass2_pipeline in LEGACY_PIPELINES:
    warnings_list.append(f"Pass 2: {pass2_model} requires 'transformers' pipeline. Auto-correcting.")
    pass2_pipeline = "transformers"

# Memory warning
heavy_enhancers = {'clearvoice', 'bs-roformer', 'zipenhancer'}
if pass1_speech_enhancer in heavy_enhancers and pass2_speech_enhancer in heavy_enhancers:
    warnings_list.append("Using GPU enhancers on both passes may cause OOM on T4. Suggest using ffmpeg-dsp for one.")

# Helpers
def build_ffmpeg_filters(amplify, loudnorm, compress, highpass):
    filters = []
    if amplify: filters.append("amplify")
    if loudnorm: filters.append("loudnorm")
    if compress: filters.append("compress")
    if highpass: filters.append("highpass")
    return ",".join(filters) if filters else None

def map_value(val):
    return None if val == "automatic" else val

def map_segmenter(val):
    return "none" if val == "none" else map_value(val)

# Build Configs
WHISPERJAV_CONFIG = {
    'use_two_step': use_two_step,
    'pass1_pipeline': pass1_pipeline,
    'pass1_sensitivity': pass1_sensitivity,
    'pass1_speech_segmenter': map_segmenter(pass1_speech_segmenter),
    'pass1_model': model_map[pass1_model],
    'pass2_pipeline': pass2_pipeline,
    'pass2_sensitivity': pass2_sensitivity,
    'pass2_speech_segmenter': map_segmenter(pass2_speech_segmenter),
    'pass2_model': model_map[pass2_model],
    'merge_strategy': combine_map[merge_method],
    'folder_name': folder_name,
    'subtitle_language': language_map[subtitle_language],
    'translation_service': translation_service,
    'local_model': local_model,
    'api_key': api_key,
    'translation_style': tone_map[translation_style],
    'opening_credit': opening_credit,
    'closing_credit': closing_credit,
    'auto_disconnect': auto_disconnect,
    # Venv paths
    'venv_path': VENV_PATH,
    'whisperjav_cmd': WHISPERJAV_CMD,
    'whisperjav_translate_cmd': WHISPERJAV_TRANSLATE_CMD,
    # Helpers for Step 2 display
    '_quality': pass1_pipeline,
    '_speech_detection': pass1_sensitivity,
    '_speech_segmenter': pass1_speech_segmenter,
    '_model': pass1_model,
    '_secondpass_quality': pass2_pipeline,
    '_merge_method': merge_method,
    '_secondpass_model': pass2_model,
    '_translation_style': translation_style,
}

WHISPERJAV_EXPERT_CONFIG = {
    'pass1_scene_detector': map_value(pass1_scene_detector),
    'pass1_speech_segmenter': map_segmenter(pass1_speech_segmenter),
    'pass1_speech_enhancer': None if pass1_speech_enhancer == "none" else pass1_speech_enhancer,
    'pass1_ffmpeg_filters': build_ffmpeg_filters(pass1_ffmpeg_amplify, pass1_ffmpeg_loudnorm, pass1_ffmpeg_compress, pass1_ffmpeg_highpass) if pass1_speech_enhancer == "ffmpeg-dsp" else None,
    'pass2_scene_detector': map_value(pass2_scene_detector),
    'pass2_speech_segmenter': map_segmenter(pass2_speech_segmenter),
    'pass2_speech_enhancer': None if pass2_speech_enhancer == "none" else pass2_speech_enhancer,
    'pass2_ffmpeg_filters': build_ffmpeg_filters(pass2_ffmpeg_amplify, pass2_ffmpeg_loudnorm, pass2_ffmpeg_compress, pass2_ffmpeg_highpass) if pass2_speech_enhancer == "ffmpeg-dsp" else None,
    '_pass1_scene_detector': pass1_scene_detector,
    '_pass1_speech_enhancer': pass1_speech_enhancer,
    '_pass2_scene_detector': pass2_scene_detector,
    '_pass2_speech_enhancer': pass2_speech_enhancer,
}

# Display Status
from IPython.display import display, HTML
for w in warnings_list:
    display(HTML(f'<div style="padding:6px 10px;background:#fef9c3;border-radius:4px;font-size:10px;margin-bottom:4px"><b>⚠️ Warning:</b> {w}</div>'))

display(HTML(f'<div style="padding:10px;background:#f0f9ff;border-radius:4px;font-size:11px">'
             f'<b>Configuration Loaded</b><br>'
             f'Mode: {"Two-Step" if use_two_step else "Standard"} | '
             f'Model: {pass1_model} ({pass1_pipeline}) | '
             f'Output: {subtitle_language}'
             f'</div>'))

In [ ]:
#@title Step 2: Transcribe { display-mode: "form" }
#@markdown Connect Drive → Install → Transcribe all media files → Add credits

import os, sys, subprocess, shlex, time
from pathlib import Path
from IPython.display import display, HTML

def status(msg, ok=True):
    icon = "✓" if ok else "✗"
    print(f"{icon} {msg}")

def section(title):
    print(f"\n{'─'*40}\n{title}\n{'─'*40}")

# Check config
if 'WHISPERJAV_CONFIG' not in dir():
    display(HTML('<div style="padding:8px;background:#fef2f2;border-radius:4px;color:#991b1b;font-size:10px"><b>Error:</b> Run Step 1 first</div>'))
    raise SystemExit()
cfg = WHISPERJAV_CONFIG
expert = WHISPERJAV_EXPERT_CONFIG if 'WHISPERJAV_EXPERT_CONFIG' in dir() else None

# ═══════════════════════════════════════════
# CONNECT GOOGLE DRIVE
# ═══════════════════════════════════════════
section("CONNECTING GOOGLE DRIVE")
try:
    from google.colab import drive
    drive.mount('/content/drive', force_remount=False)
    folder_path = Path(f"/content/drive/MyDrive/{cfg['folder_name']}")
    folder_path.mkdir(parents=True, exist_ok=True)
    status(f"Connected: {folder_path}")
except Exception as e:
    status(f"Failed to connect: {e}", False)
    raise SystemExit("Google Drive connection failed")

# ═══════════════════════════════════════════
# CHECK GPU
# ═══════════════════════════════════════════
section("CHECKING GPU")
gpu_check = subprocess.run(["nvidia-smi", "--query-gpu=name,memory.total", "--format=csv,noheader"], capture_output=True, text=True)
if gpu_check.returncode != 0 or not gpu_check.stdout.strip():
    status("No GPU detected. Go to Runtime → Change runtime type → T4 GPU", False)
    raise SystemExit("No GPU detected")
status(f"GPU: {gpu_check.stdout.strip()}")

# ═══════════════════════════════════════════
# INSTALL WHISPERJAV
# ═══════════════════════════════════════════
section("INSTALLING WHISPERJAV")
install_start = time.time()

REPO_URL = "https://github.com/meizhong986/WhisperJAV.git"
REPO_PATH = "/content/WhisperJAV"
SCRIPT_PATH = f"{REPO_PATH}/installer/install_colab.sh"

# Check if already installed
venv_python = f"{cfg['venv_path']}/bin/python"
if os.path.exists(venv_python):
    check = subprocess.run([venv_python, "-c", "import whisperjav"], capture_output=True)
    if check.returncode == 0:
        status("WhisperJAV already installed (skipping)")
    else:
        status("Existing venv corrupt, reinstalling...")
        subprocess.run(["rm", "-rf", cfg['venv_path']], capture_output=True)
        if not os.path.exists(REPO_PATH):
            subprocess.run(["git", "clone", REPO_URL, REPO_PATH], capture_output=True)
        result = subprocess.run(["bash", SCRIPT_PATH],
            env={**os.environ, "PATH": f"{os.environ.get('PATH', '')}:{os.path.expanduser('~/.local/bin')}"})
        if result.returncode != 0:
            status("Installation failed", False)
            raise SystemExit("Installation failed")
else:
    # Fresh install: clone repo and run installer
    print("Installing WhisperJAV (uv-accelerated)...\n")
    if not os.path.exists(REPO_PATH):
        print(f"Cloning {REPO_URL}...")
        result = subprocess.run(["git", "clone", REPO_URL, REPO_PATH], capture_output=True, text=True)
        if result.returncode != 0:
            status(f"Failed to clone: {result.stderr}", False)
            raise SystemExit("Clone failed")
    
    if not os.path.exists(SCRIPT_PATH):
        status(f"Install script not found at {SCRIPT_PATH}", False)
        raise SystemExit("Install script missing from repo")
    
    result = subprocess.run(["bash", SCRIPT_PATH],
        env={**os.environ, "PATH": f"{os.environ.get('PATH', '')}:{os.path.expanduser('~/.local/bin')}"})
    if result.returncode != 0:
        status("Installation failed", False)
        raise SystemExit("Installation failed")

# Install speech enhancer packages if needed
if expert:
    extra_packages = set()
    for enhancer in [expert.get('pass1_speech_enhancer'), expert.get('pass2_speech_enhancer')]:
        if enhancer == 'clearvoice':
            extra_packages.add('clearvoice')
        elif enhancer == 'bs-roformer':
            extra_packages.add('bs-roformer-infer')

    if extra_packages:
        pkg_list = ' '.join(extra_packages)
        uv_path = os.path.expanduser("~/.local/bin/uv")
        if os.path.exists(uv_path):
            result = subprocess.run(f"{uv_path} pip install --python {venv_python} {pkg_list}", shell=True, capture_output=True, text=True)
        else:
            result = subprocess.run(f"{venv_python} -m pip install -q {pkg_list}", shell=True, capture_output=True, text=True)
        if result.returncode != 0:
            status(f"Speech enhancer packages failed (continuing anyway)", False)
        else:
            status(f"Speech enhancer packages ({', '.join(extra_packages)})")

status(f"Installation complete ({time.time()-install_start:.0f}s)")

# ═══════════════════════════════════════════
# FIND MEDIA FILES
# ═══════════════════════════════════════════
section("SCANNING FILES")
video_types = {'.mp4', '.mkv', '.avi', '.mov', '.wmv', '.flv', '.webm', '.m4v', '.mp3', '.wav', '.flac', '.m4a'}
videos = [f for f in folder_path.iterdir() if f.suffix.lower() in video_types]

if not videos:
    status(f"No media files in {cfg['folder_name']}/", False)
    raise SystemExit("No media files found")

status(f"Found {len(videos)} file(s)")
for v in videos[:5]:
    print(f"  • {v.name}")
if len(videos) > 5:
    print(f"  ... and {len(videos)-5} more")

existing_srts = set(folder_path.glob('*.srt'))

# ═══════════════════════════════════════════
# TRANSCRIBE
# ═══════════════════════════════════════════
section("TRANSCRIBING")

cmd = [cfg['whisperjav_cmd'], str(folder_path), '--output-dir', str(folder_path)]

if cfg['use_two_step']:
    cmd.extend(['--ensemble',
        '--pass1-pipeline', cfg['pass1_pipeline'],
        '--pass1-sensitivity', cfg['pass1_sensitivity'],
        '--pass2-pipeline', cfg['pass2_pipeline'],
        '--pass2-sensitivity', cfg['pass2_sensitivity'],
        '--merge-strategy', cfg['merge_strategy']])

    if cfg['pass1_speech_segmenter']:
        cmd.extend(['--pass1-speech-segmenter', cfg['pass1_speech_segmenter']])
    if cfg['pass2_speech_segmenter']:
        cmd.extend(['--pass2-speech-segmenter', cfg['pass2_speech_segmenter']])
    if cfg['pass1_model']:
        cmd.extend(['--pass1-model', cfg['pass1_model']])
    if cfg['pass2_model']:
        cmd.extend(['--pass2-model', cfg['pass2_model']])

    if expert:
        if expert.get('pass1_scene_detector'):
            cmd.extend(['--pass1-scene-detector', expert['pass1_scene_detector']])
        if expert.get('pass1_speech_enhancer'):
            if expert['pass1_speech_enhancer'] == 'ffmpeg-dsp':
                effects = expert.get('pass1_ffmpeg_filters') or 'loudnorm'
                cmd.extend(['--pass1-speech-enhancer', f'ffmpeg-dsp:{effects}'])
            else:
                cmd.extend(['--pass1-speech-enhancer', expert['pass1_speech_enhancer']])
        if expert.get('pass2_scene_detector'):
            cmd.extend(['--pass2-scene-detector', expert['pass2_scene_detector']])
        if expert.get('pass2_speech_enhancer'):
            if expert['pass2_speech_enhancer'] == 'ffmpeg-dsp':
                effects = expert.get('pass2_ffmpeg_filters') or 'loudnorm'
                cmd.extend(['--pass2-speech-enhancer', f'ffmpeg-dsp:{effects}'])
            else:
                cmd.extend(['--pass2-speech-enhancer', expert['pass2_speech_enhancer']])

    p1_info = cfg['_quality']
    if cfg['_speech_segmenter'] != 'automatic':
        p1_info += f" + {cfg['_speech_segmenter']}"
    if cfg['_model'] != 'automatic':
        p1_info += f" ({cfg['_model']})"
    if expert:
        if expert.get('_pass1_scene_detector') != 'automatic':
            p1_info += f" [scene:{expert['_pass1_scene_detector']}]"
        if expert.get('_pass1_speech_enhancer') != 'none':
            p1_info += f" [enh:{expert['_pass1_speech_enhancer']}]"

    p2_info = cfg['_secondpass_quality']
    if cfg.get('_secondpass_model', 'automatic') != 'automatic':
        p2_info += f" ({cfg['_secondpass_model']})"
    if expert:
        if expert.get('_pass2_scene_detector') != 'automatic':
            p2_info += f" [scene:{expert['_pass2_scene_detector']}]"
        if expert.get('_pass2_speech_enhancer') != 'none':
            p2_info += f" [enh:{expert['_pass2_speech_enhancer']}]"

    print(f"Mode: Two-Step")
    print(f"  Pass 1: {p1_info}")
    print(f"  Pass 2: {p2_info}")
    print(f"  Merge: {cfg['_merge_method']}")
else:
    cmd.extend(['--mode', cfg['pass1_pipeline'], '--sensitivity', cfg['pass1_sensitivity']])
    if cfg['pass1_speech_segmenter']:
        cmd.extend(['--speech-segmenter', cfg['pass1_speech_segmenter']])
    if cfg['pass1_model']:
        cmd.extend(['--model', cfg['pass1_model']])
    if expert and expert.get('pass1_scene_detector'):
        cmd.extend(['--scene-detection-method', expert['pass1_scene_detector']])

    mode_info = f"{cfg['_quality']}/{cfg['_speech_detection']}"
    if cfg['_speech_segmenter'] != 'automatic':
        mode_info += f" + {cfg['_speech_segmenter']}"
    if cfg['_model'] != 'automatic':
        mode_info += f" ({cfg['_model']})"
    if expert and expert.get('_pass1_scene_detector') != 'automatic':
        mode_info += f" [scene:{expert['_pass1_scene_detector']}]"
    print(f"Mode: Standard ({mode_info})")

if cfg['subtitle_language'] == 'direct-to-english':
    cmd.extend(['--subs-language', 'direct-to-english'])
    print(f"Output: English (Whisper auto-translate)")
else:
    cmd.extend(['--subs-language', 'native'])
    if cfg['subtitle_language'] == 'llm':
        print(f"Output: Japanese (AI translation will follow in Step 3)")
    else:
        print(f"Output: Japanese")

print(f"Input: {folder_path}\n")

full_cmd = shlex.join(cmd)
process = subprocess.Popen(full_cmd, shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, bufsize=1, universal_newlines=True)
for line in process.stdout:
    print(line, end='')
process.wait()

if process.returncode != 0:
    status("Transcription failed", False)
    raise SystemExit("Transcription failed")

all_srts = set(folder_path.glob('*.srt'))
new_srts = list(all_srts - existing_srts)
new_srts.sort(key=lambda x: x.name)

WHISPERJAV_NEW_SRTS = new_srts
WHISPERJAV_FOLDER_PATH = folder_path

status(f"Created {len(new_srts)} new subtitle file(s)")

# ═══════════════════════════════════════════
# ADD CREDITS
# ═══════════════════════════════════════════
section("ADDING CREDITS")

if cfg['opening_credit'] or cfg['closing_credit']:
    credits_count = 0
    for srt_file in new_srts:
        try:
            content = srt_file.read_text(encoding='utf-8')
            if cfg['opening_credit']:
                content = f"0\n00:00:00,000 --> 00:00:00,500\n{cfg['opening_credit']}\n\n" + content
            if cfg['closing_credit']:
                content += f"\n9999\n23:59:58,000 --> 23:59:59,000\n{cfg['closing_credit']}\n"
            srt_file.write_text(content, encoding='utf-8')
            credits_count += 1
        except Exception as e:
            print(f"  Warning: Could not add credits to {srt_file.name}: {e}")
    status(f"Credits added to {credits_count} file(s)")
else:
    status("No credits configured")

# ═══════════════════════════════════════════
# TRANSCRIPTION COMPLETE
# ═══════════════════════════════════════════
section("TRANSCRIPTION COMPLETE")

if cfg['subtitle_language'] == 'llm' and (cfg['api_key'] or cfg['translation_service'] == 'local'):
    display(HTML(f'<div style="padding:8px 10px;background:#fef9c3;border-radius:4px;border-left:2px solid #ca8a04;font-size:10px"><b>✓ Transcription done!</b> {len(new_srts)} file(s). Run Step 3 next to start AI Translation.</div>'))
else:
    display(HTML(f'<div style="padding:8px 10px;background:#f0fdf4;border-radius:4px;border-left:2px solid #16a34a;font-size:10px"><b>✓ Done!</b> {len(new_srts)} subtitle(s) saved to Google Drive/{cfg["folder_name"]}/</div>'))
    if cfg['subtitle_language'] == 'llm' and not cfg['api_key'] and cfg['translation_service'] != 'local':
        print("Note: AI translation skipped (no API key provided)")

    if cfg['auto_disconnect']:
        print("\nAuto-disconnecting in 10s to save GPU credits...")
        time.sleep(10)
        try:
            from google.colab import runtime
            runtime.unassign()
        except: pass

In [ ]:
#@title Step 3: AI Translation { display-mode: "form" }
#@markdown Translate the newly generated SRTs using `whisperjav-translate` (runs only if you selected **English (AI translate)** in Step 1).

import os, shlex, subprocess, time
from pathlib import Path
from IPython.display import display, HTML

def status(msg, ok=True):
    icon = "✓" if ok else "✗"
    print(f"{icon} {msg}")

def section(title):
    print(f"\n{'─'*40}\n{title}\n{'─'*40}")

# Preconditions
if 'WHISPERJAV_CONFIG' not in dir():
    display(HTML('<div style="padding:8px;background:#fef2f2;border-radius:4px;color:#991b1b;font-size:10px"><b>Error:</b> Run Step 1 first</div>'))
    raise SystemExit()
if 'WHISPERJAV_NEW_SRTS' not in dir() or 'WHISPERJAV_FOLDER_PATH' not in dir():
    display(HTML('<div style="padding:8px;background:#fef2f2;border-radius:4px;color:#991b1b;font-size:10px"><b>Error:</b> Run Step 2 first</div>'))
    raise SystemExit()

cfg = WHISPERJAV_CONFIG
new_srts = WHISPERJAV_NEW_SRTS
folder_path = WHISPERJAV_FOLDER_PATH

# Check if AI translation is needed
if cfg['subtitle_language'] != 'llm':
    display(HTML('<div style="padding:8px 10px;background:#f0f9ff;border-radius:4px;border-left:2px solid #3b82f6;font-size:10px"><b>ℹ Skipped:</b> AI translation not selected</div>'))
    raise SystemExit()

# Check API key requirement (not needed for local provider)
is_local = cfg['translation_service'] == 'local'
if not is_local and not cfg['api_key']:
    display(HTML('<div style="padding:8px;background:#fef2f2;border-radius:4px;color:#991b1b;font-size:10px"><b>Error:</b> No API key provided for cloud translation. Use "local" provider for free GPU translation.</div>'))
    raise SystemExit()

if not new_srts:
    display(HTML('<div style="padding:8px;background:#fef2f2;border-radius:4px;color:#991b1b;font-size:10px"><b>Error:</b> No subtitle files to translate</div>'))
    raise SystemExit()

# Set up API key env var for provider (not needed for local)
if not is_local:
    env_map = {
        "deepseek": "DEEPSEEK_API_KEY",
        "openrouter": "OPENROUTER_API_KEY",
        "gemini": "GEMINI_API_KEY",
        "claude": "ANTHROPIC_API_KEY",
        "gpt": "OPENAI_API_KEY",
    }
    os.environ[env_map.get(cfg['translation_service'], "API_KEY")] = cfg['api_key']

section("AI TRANSLATION")
if is_local:
    print(f"Provider: local ({cfg.get('local_model', 'gemma-9b')})")
    print("Note: First run downloads model (~5GB)")
else:
    print(f"Provider: {cfg['translation_service']}")
print(f"Style: {cfg['_translation_style']}")
print(f"Files to translate: {len(new_srts)}\n")

translated_files = []
failed_files = []

for i, srt_file in enumerate(new_srts, 1):
    print(f"[{i}/{len(new_srts)}] Translating: {srt_file.name}")

    # Use venv path for translate command
    translate_cmd = [
        cfg['whisperjav_translate_cmd'],
        '-i', str(srt_file),
        '--provider', cfg['translation_service'],
        '-t', 'english',
        '--tone', cfg['translation_style'],
        '--stream',
    ]

    if is_local:
        translate_cmd.extend(['--model', cfg.get('local_model', 'gemma-9b')])

    full_cmd = shlex.join(translate_cmd)

    try:
        process = subprocess.Popen(
            full_cmd,
            shell=True,
            stdout=subprocess.PIPE,
            stderr=subprocess.PIPE,
            bufsize=1,
            universal_newlines=True,
        )

        for line in process.stderr:
            print(f"    {line}", end='')

        stdout_output, _ = process.communicate()

        if process.returncode == 0:
            output_path = stdout_output.strip()
            if output_path:
                translated_files.append(Path(output_path))
            status(f"Completed: {srt_file.name}")
        else:
            status(f"Failed: {srt_file.name}", False)
            failed_files.append(srt_file)

    except Exception as e:
        status(f"Error translating {srt_file.name}: {e}", False)
        failed_files.append(srt_file)

    print()

section("COMPLETE")
if failed_files:
    display(HTML(f'<div style="padding:8px 10px;background:#fef9c3;border-radius:4px;border-left:2px solid #ca8a04;font-size:10px"><b>⚠ Partially done!</b> {len(translated_files)}/{len(new_srts)} translated. {len(failed_files)} failed.</div>'))
else:
    display(HTML(f'<div style="padding:8px 10px;background:#f0fdf4;border-radius:4px;border-left:2px solid #16a34a;font-size:10px"><b>✓ All done!</b> {len(new_srts)} Japanese + {len(translated_files)} English subtitle(s) in Google Drive/{cfg["folder_name"]}/</div>'))

if cfg.get('auto_disconnect'):
    print("\nAuto-disconnecting in 10s to save GPU credits...")
    time.sleep(10)
    try:
        from google.colab import runtime
        runtime.unassign()
    except: pass
else:
    print("\nRemember to disconnect manually to save GPU credits.")